In [2]:
import re
import spacy
import os
import shutil
from pathlib import Path
nlp = spacy.load("pl_core_news_lg")

In [34]:
"""
Ważna uwaga:
Pliki z lekturami (trzeba funkcji ustawić argument is_fragment=False) do wczytania muszą mieć następującą strukturę:
Autor

Tytuł(dowolna ilość linii)

Ewentualny numer ISBN

Tekst(dowolna ilość linii)
Ewentualna stopka po treście(przypis od wolnychlektur): -----
"""
"""
Przy podawaniu fragmentu(is_fragment=True) skanowany jest cały plik, więc struktura tekstu jest dowolna
"""
"""
tworzy worek słów z pliku
file_name - nazwa pliku z której utworzyc worek słów; with_stop_words - True/False mówiący czy usuwać stop wordsy czy nie
is_fragment - True/False - mówi czy podawany plik to fragment do sprawdzenia, czy cały utwór
"""
def create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True, only_stop_words=False):
#struktura worka: indeks to indeks, elementem jest lista ze słowem i liczebnością
    bag = {}
    with open(file_name,'r', encoding='utf-8') as file:
#autor
        if not is_fragment:
            bag[0] = get_author(file)
            
        for line in file:
            for word in line.split():
#sprawdzanie czy linia nie jest stopką. Jak jest-skończ działanie
                if not is_fragment and word == "-----":
                    return bag
            bag = create_bag_of_words_from_string(line, bag, with_stop_words, lemma, only_stop_words)
    return bag

def create_bag_of_words_from_string(string, bag, with_stop_words=False, lemma=True, only_stop_words=False):
    #struktura worka: indeks to indeks, elementem jest lista ze słowem i liczebnością
    stop_words = []
#tablica znaków interpunkcyjnych do pozbycia się z wyrazów
    punc = '''…!”„()-[]»{};:'"\,<>./?@#$%^&*_~«—'''+ "\n" + " "
#ewentualne przypisanie stopwordsów
    if not with_stop_words or only_stop_words:
        stop_words = get_stop_words_from_file('stopWords.txt')
    words = words_from_line(string, lemma, punc)
    for word in words:
        word = word.lower()
#jeśli jest stop wordsem- nie uwzględniamy (jeśli w funkcji było True, to 'stop_words' jest puste, czyli nie wejdzie do ifa)    
        if word in stop_words:
            if only_stop_words:
                bag = add_to_bag(bag, word)
            continue
        elif not only_stop_words:
            bag = add_to_bag(bag, word)
    return bag

def add_to_bag(bag, word):
    if (word not in bag) and (len(word) != 0):
        bag[word] = 1
#inkrementacja liczności słowa, jeśli już istnieje w worku
    elif len(word) != 0:
        bag[word] += 1
    return bag

"""
Tworzy tablicę słów w zależności od tego, czy używamy lematyzacji
- Jeżeli nie używamy lematyzacji to usuwa ze słów ewentualne znaki interpunkcyjne
- Jeżeli używamy lematyzacji to usuwa nadmierne spacje, generuje tokeny i zamienia je na słowa
w wersji podstawowej zależnie od kontekstu w linijce. Pomija słowa będące znakami interpunkcyjnymi
"""
def words_from_line(line, lemma, punc):
    words = []
    if not lemma:
        for word in line.split():
            for ele in word:
                if ele in punc:
                    word = word.replace(ele, "")
            words.append(word)
    
    else:
        line = re.sub(' +', ' ', line)
        tokens = nlp(line, disable=["parser", "ner"])
        for token in tokens:
            word = token.lemma_
            if word in punc:
                continue
            words.append(word)
        
    return words

"""
word_bags - lista stworzonych z lektur worków słów
WAŻNE: w każdym worku pod indeksem 0(liczba, nie string) musi znajdować się autor danego utworu
WAŻNE2: używać przed użyciem funkcji count_percents na workach
struktura outputu: słownik, gdzie kluczem są autorzy, a wartościami worek słów(z których zostanie wyrzucony klucz 0-z autorem)
"""
def combine_word_bags(word_bags):
    combined = {}
#przechodzenie po wszystkich workach z listy
    for bag in word_bags:
        keys = combined.keys()
#pobieranie autora z aktualnego worka i usuwanie go ze słownika
        author = bag[0]
        del bag[0]
#jeśli autor już jest w wynikowym worku, to aktualizujemy wystąpienia słów u niego
        if author in keys:
            for elem in bag:
#jeśli autor ma już dane słowo, to dodajemy ilość wystąpień z aktualnie iterowanego worka
                if elem in combined[author]:
                    combined[author][elem] += bag[elem]
#jeśli słowa jeszcze nie ma, to dodajemy rekord
                else:
                    combined[author][elem] = bag[elem]
#jeśli autora nie ma jeszcze w wynikowym worku, to dodajemy rekord, przypisując do niego aktualny worek
        else:
            combined[author] = bag
    return combined

"""
funkcja zmienia ilosć wystąpień danego słowa na procent wartość wystąpień w tekście
"""
def count_percents(word_bag):
    amount = 0
#zliczanie ilości słów w worku
    for elem in word_bag:
        if elem == 0:
            continue
        amount += word_bag[elem]
#przeliczanie wystąpień na wartości procentowe
    for elem in word_bag:
        if elem == 0:
            continue
        word_bag[elem] = word_bag[elem]/amount*100
    return word_bag

"""
Funkcja zapisuje worki słów do plików. Folderem jest BagsOfWords/, gdzie pliki tekstowe mają tytuł autor_worka.txt.
Format pojedyńczej linii z rekordem: słowo ilość_wystąpień
word_bags- słownik worków, gdzie kluczem jest autor, a wartością worek
"""
def save_word_bags_to_file(word_bags, directory):
    if not os.path.isdir('Bags'):
        os.mkdir('Bags')
    os.mkdir(directory)
    for key in word_bags.keys():
        bag = word_bags[key]
        file = open(directory + key + ".txt", "w", encoding="utf-8")
        for b_keys in bag.keys():
            file.write(b_keys + " " + str(bag[b_keys]) + '\n')
        file.close()
      
"""
Funkcja pobiera już wcześniej utworzone worki słów z folderu
directory- folder z workami słów 
WAŻNE:
directory podawać bez ukośnika! Czysta nazwa!
pliki z workami słów danych autorów muszą mieć za nazwę imię i nazwisko tego autora
"""
def get_word_bags_from_directory(directory):
    files = os.listdir(directory)
    bags = {}
    for author in files:
        tmp_bag = {}
        file = open(directory + "\\" + author, encoding='utf-8')
        for line in file:
            line = line.split()
            tmp_bag[line[0]] = int(line[1])
        bags[repr(author).replace(".txt", "").replace("'", "")] = tmp_bag
    return bags
    
            

In [35]:
#To są różne funkcje pomocnicze
"""
funkcja pobiera stop wordsy z pliku i zwraca je jako listę
"""
def get_stop_words_from_file(file_name):
    stops = []
    borders = ["\\n", "'"]
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            single_line = repr(line)
            for char in borders:
                single_line = single_line.replace(char,"")
            if not(single_line == ""):
                stops.append(single_line)
    return stops

"""
funkcja pobiera autora utworu, przeskakuje tytuł i przeskakuje number ISBN
"""
def get_author(file):
    author = file.readline()
    author = author.replace("\n","")
    author = list(author)
    if author[0] == " ":
        author[0] = ""
    author = "".join(author)
    
    file.readline()
    while not (file.readline()  == "\n"):
        continue
    position_before = file.tell()
    if not file.readline().startswith('ISBN'):
        file.seek(position_before)
    return author

In [42]:
"""
Wywołuje sięją po to, aby uzyskać worki słów dla autorów- uprzednio zapisanych lub stworzone na nowo
Metoda pomocnicza - automatycznie tworzy worki słów (od parametru combined zależy czy będą to 
worki słów każdej książki, czy wspólne dla każdego autora)
"""
def create_bags_of_words_from_authors(only_stop_words, with_stop_words, lemma):
    bags = {}
#tworzenie ścieżki folderu na podstawie argumentów
    path = "Bags/BagOfWords_"
    if only_stop_words:
        path += "only"
    elif with_stop_words:
        path += "with"
    elif not with_stop_words:
        path += "no"
    path += "StopWords_"
    if not lemma:
        path += "Not"
    path += "Lemma/"
#Jeśli taki folder jużistnieje, oznacza, że jużwcześniej go utworzyliśmy i tylko wczytujemy gotowe worki słów
    if os.path.isdir(path):
        bags = get_word_bags_from_directory(path)
#Jeśli nie ma takiego folderu, to trzeba utworzyć worki
    else:
        books_list = os.listdir('Books')
        word_bags = []
        for book in books_list:
            print("doing " + book)
            word_bags.append(create_bag_of_words_from_file('Books/'+book, False, with_stop_words, lemma, only_stop_words))
        bags = combine_word_bags(word_bags)
        save_word_bags_to_file(bags, path)
    return bags
"""
def create_bags_of_words_from_authors(combined=True, with_stop_words=False, lemma=True):
    books_list = os.listdir('Books')
    word_bags = []
    for book in books_list:
        word_bags.append(create_bag_of_words_from_file('Books/'+book, with_stop_words, lemma))            
    if combined:
        word_bags = combine_word_bags(word_bags)            
    return word_bags
"""

"\ndef create_bags_of_words_from_authors(combined=True, with_stop_words=False, lemma=True):\n    books_list = os.listdir('Books')\n    word_bags = []\n    for book in books_list:\n        word_bags.append(create_bag_of_words_from_file('Books/'+book, with_stop_words, lemma))            \n    if combined:\n        word_bags = combine_word_bags(word_bags)            \n    return word_bags\n"

In [37]:
#Tak się tego używa do wczytania worków
###########################################
#bag = create_bags_of_words_from_authors(False, False, True)
#############################

In [41]:
#create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True, only_stop_words=False):
ab = create_bag_of_words_from_file('panTadeusz.txt', False, False, True, True)
print(ab)

{0: 'Adam Mickiewicz', 'się': 1497, 'w': 1757, 'u': 117, 'o': 348, 'nad': 112, 'i': 2128, 'na': 1304, 'moja': 19, 'ty': 163, 'jak': 579, 'ile': 14, 'trzeba': 38, 'ten': 131, 'tylko': 149, 'kto': 93, 'dziś': 80, 'cały': 133, 'bo': 196, 'po': 323, 'co': 289, 'z': 1482, 'on': 1186, 'ja': 441, 'do': 565, 'gdy': 189, 'od': 244, 'pod': 125, 'twój': 13, 'za': 335, 'tak': 312, 'my': 148, 'tych': 34, 'gdzie': 94, 'a': 565, 'wszystko': 67, 'jakby': 67, 'przed': 104, 'lecz': 206, 'tym': 107, 'że': 555, 'przy': 100, 'nie': 933, 'zapewne': 11, 'dobrze': 51, 'wszystkich': 40, 'właśnie': 38, 'sam': 164, 'jako': 145, 'też': 61, 'mało': 19, 'niż': 26, 'tu': 142, 'był': 136, 'albo': 60, 'daleko': 36, 'obok': 17, 'już': 313, 'nawet': 71, 'by': 79, 'tej': 63, 'to': 531, 'były': 23, 'jeden': 146, 'ale': 176, 'było': 38, 'jeszcze': 101, 'bez': 95, 'była': 48, 'aż': 118, 'znowu': 71, 'nigdy': 41, 'więc': 112, 'przez': 101, 'nim': 29, 'coś': 30, 'być': 275, 'czy': 136, 'według': 1, 'aby': 24, 'pan': 298, 'któ

In [6]:
#string = """przykłądowy string w celu testu metody aaaa lisa
#nanana ptak leci nad ptakiem i ptak leci z ptakiem obok lisowi lis lis jest zły lisie lis ma bardzo ostre dzik"""
#aaa = create_bag_of_words_from_string(string, {}, True)
#print(aaa)
#create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True)
#aaa = create_bag_of_words_from_file("panTadeusz.txt", False, True, True)
#print("")
#print(aaa)

In [7]:
#to jest do moich testów, ale tak to w sumie będzie juzpotem wczytywane
"""
books_list = os.listdir('Books')
word_bags = []
for book in books_list:
    word_bags.append(create_bag_of_words_from_file('Books/'+book, False, False, False))
word_bags = combine_word_bags(word_bags)
print(len(word_bags))
for k in word_bags.keys():
    print(k)
for k in word_bags.keys():
    word_bags[k] = count_percents(word_bags[k])
save_word_bags_to_file(word_bags)
"""
#########################################
#Tutaj sytuacja, gdy uruchamiamy po raz kolejny i już mamy gotowe, zapisane worki słów
#dicti = get_word_bags_from_directory("BagOfWords")


"\nbooks_list = os.listdir('Books')\nword_bags = []\nfor book in books_list:\n    word_bags.append(create_bag_of_words_from_file('Books/'+book, False, False, False))\nword_bags = combine_word_bags(word_bags)\nprint(len(word_bags))\nfor k in word_bags.keys():\n    print(k)\nfor k in word_bags.keys():\n    word_bags[k] = count_percents(word_bags[k])\nsave_word_bags_to_file(word_bags)\n"